# Restart
데이터셋 분류를 잘못했다. <br/>

test에 포함된 폰트의 글자가 train, val에도 섞여들었음을 확인했다. <br/>
생성한 이미지에서 '폰트'를 기준으로 train / val / test 개수를 설정한 후 별도의 폴더에 저장하자.
<br/>
**obj 파일을 만들기 전에 나누는 것, 이미지 파일이 아닌 폰트를 기준으로 개수를 나누는 것** 이 핵심이다.

---

또한 test셋을 출력할 때에는 같은 폰트(카테고리 말고!)끼리 묶여서 출력하도록 sorting하자.<br/>
- a\~Z 이미지를 만들어내기 전에, 폰트별로 먼저 split하고 a\~Z를 만들자.


---

In [2]:
import os
import glob
import shutil
import numpy as np
from tqdm import tqdm
from PIL import Image,ImageDraw,ImageFont
import torch
import matplotlib.pyplot as plt
%matplotlib inline

---

## 이미지 파일 split
train / val / test = 0.8 / 0.15 / 0.05 비율을 폰트 파일에 적용하여 나누자 <br/>
7번 노트북의 코드를 사용할 것이다. <br/>
이전과 같이, **serif, display, handwriting** 카테고리만 생성한다.

In [1]:
r_train = 0.8
r_val   = 0.15
r_test  = 0.05

In [3]:
def extract_filename(path):
    filenames = []
    for filename in glob.iglob(path + '*.png'):
        filenames.append(filename[17:])
    return filenames

In [4]:
# 원본 이미지 파일명
original_serif       = extract_filename('collection/img/0/')
original_display     = extract_filename('collection/img/2/')
original_handwriting = extract_filename('collection/img/3/')

In [5]:
# 카테고리별 파일 개수
size_serif       = int(len(original_serif)/52)       # 182 
size_display     = int(len(original_display)/52)     # 283 
size_handwriting = int(len(original_handwriting)/52) # 140 

In [6]:
def split_each_font(size, splt_test, splt_val):
    test = int(np.floor(splt_test * size))
    val  = int(np.floor((splt_test + splt_val) * size))
    return test, val

In [7]:
# serif split
split_test_serif, split_valid_serif = split_each_font(size_serif, r_test, r_val)

# display split
split_test_display, split_valid_display = split_each_font(size_display, r_test, r_val)

# serif split
split_test_handwriting, split_valid_handwriting = split_each_font(size_handwriting, r_test, r_val)

In [8]:
def get_idx_sampler(size, splt_test, splt_val):
    idx = list(range(size))
    train_idxs = idx[splt_val:]
    val_idxs   = idx[splt_test: splt_val]
    test_idxs  = idx[: splt_test]
    return train_idxs, val_idxs, test_idxs

In [9]:
train_i_serif, val_i_serif, test_i_serif = get_idx_sampler(size_serif, split_test_serif, split_valid_serif)
train_i_disp, val_i_disp, test_i_disp    = get_idx_sampler(size_display, split_test_display, split_valid_display)
train_i_hand, val_i_hand, test_i_hand    = get_idx_sampler(size_handwriting, split_test_handwriting, split_valid_handwriting)

In [10]:
print('           train val test')
print('serif:       ', len(train_i_serif), len(val_i_serif), len(test_i_serif))
print('display:     ', len(train_i_disp), len(val_i_disp), len(test_i_disp))
print('handwriting: ', len(train_i_hand), len(val_i_hand), len(test_i_hand))

           train val test
serif:        146 27 9
display:      227 42 14
handwriting:  112 21 7


위 갯수는 '폰트의 개수'를 의미한다. 이제 이미지가 담긴 폴더에서 해당 갯수만큼 추출해 별도의 폴더로 옮기자

---
## Separate folders


In [16]:
from_dir = 'collection/img/' # + index
to_dir = 'collection/img/'   # + train/val/test

In [12]:
len(glob.glob('collection/img/0/*.png'))

9464

In [13]:
def move_img(cateIdx, purpose, fromDir, toDir, filename):
    fromDir += str(cateIdx) + '/' + filename
    toDir   += purpose + '/' + filename
    shutil.move(fromDir, toDir)

In [23]:
categories   = [0, 2, 3] # serif, display, handwriting
purposes     = ['train', 'val', 'test']

In [21]:
dict_params = {
    0 : {
        'train': len(train_i_serif),
        'val'  : len(val_i_serif),
        'test' : len(test_i_serif)
    },
    2 : {
        'train': len(train_i_disp),
        'val'  : len(val_i_disp),
        'test' : len(test_i_disp)
    },
    3 : {
        'train': len(train_i_hand),
        'val'  : len(val_i_hand),
        'test' : len(test_i_hand)
    },
}

안전하게 따로 작업하자...ㅎ

In [86]:
# spliting - Serif
for purpose in purposes: # train, val, test
    for (idx, img) in enumerate(sorted(glob.glob('collection/img/{}/*.png'.format(0)))): # sorted를 하면 정렬된다는 것을 잊고 살았다.
        img_name = img[17:]
        num_file = dict_params[0][purpose] * 52
        move_img(0, purpose, from_dir, to_dir, img_name)
        if idx+1 == num_file:
            break

In [91]:
# spliting - Display
for purpose in purposes: # train, val, test
    for (idx, img) in enumerate(sorted(glob.glob('collection/img/{}/*.png'.format(2)))):
        img_name = img[17:]
        num_file = dict_params[2][purpose] * 52
        move_img(2, purpose, from_dir, to_dir, img_name)
        if idx+1 == num_file:
            break

In [95]:
# spliting - Handwriting
for purpose in purposes: # train, val, test
    for (idx, img) in enumerate(sorted(glob.glob('collection/img/{}/*.png'.format(3)))):
        img_name = img[17:]
        num_file = dict_params[3][purpose] * 52
        move_img(3, purpose, from_dir, to_dir, img_name)
        if idx+1 == num_file:
            break

In [103]:
# 개수 확인
print(len(glob.glob('collection/img/train/*.png'))/52) # 146 + 227 + 112 = 485개 폰트
print(len(glob.glob('collection/img/val/*.png'))/52)   # 27 + 42 + 21 = 90개 폰트
print(len(glob.glob('collection/img/test/*.png'))/52)  # 9 + 14 + 7 = 

485.0
90.0
30.0
